## Регресия
### Генерация данных
Для демонстрации решения задачи регрессии при помощи решающего дерева сгенерируем данные, в которых есть один целевой признак и один нецелевой. В этом случае имеем дело с задачей одномерной регрессии и решением будет некоторая линия. В файле scikit-learn_regression_example.ipynb решалась задача линейной регресси, в которой решением служилая прямая линия. Мы же для разнообразия сгенерируем данные, в которых зависимость целевого признака от нецелевого будет носить нелинейный характер. Пусть зависимость будет параболической.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Функция `np.random.seed()` позволяет установить начальное состояние генератора случайных чисел, что позволяет получить одни и те же результаты при повторном запуске программы. Если вы используете модуль `np.random` без установки начального состояния, то каждый раз, когда вы запускаете программу, вы будете получать разные случайные числа.

Функция `np.random.rand()` создает массив заданной формы, содержащий случайные значения, равномерно распределенные между 0 и 1. Если не задать форму в аргументах функции, то вместо массива будет возвращено число. 

Функция `np.random.randn()` создает массив заданной формы, содержащий случайные значения, c нормальным распределеннием (среднее значение равно 0, стандартное отклонение равно 1). Если не задать форму в аргументах функции, то вместо массива будет возвращено число. 

In [ ]:
# Quadratic training set + noise
np.random.seed(42)
n = 200                     # число точек

# генерация значений нецелевого признака
X = np.random.rand(n, 1)    
print(f"{X.shape = }")

# генерация значений целевого признака
y = 4 * (X - 0.5) ** 2
print(f"{y.shape = }")

# добавления шума
y = y + np.random.randn(n, 1) / 10

# визуализация
plt.plot(X, y, "b.")
plt.show()

## Обучение деревье различной глубины

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

tree_reg1 = DecisionTreeRegressor(max_depth=1)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

In [ ]:
plot_tree(tree_reg1, filled=True, rounded=True)
plt.show()

In [ ]:
plt.figure(figsize=(16, 16))
plot_tree(tree_reg2, filled=True)
plt.show()

#### Визуализация линии регрессии
Напишем функцию, которая будет визуализировать точки данных и решающую линию.

In [ ]:
def plot_regression_predictions(model, X, y, axes=[0, 1, -0.2, 1.2]):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = model.predict(x1)
    plt.axis(axes)
    plt.xlabel('$x_1$', fontsize=18)
    plt.ylabel('$y$', fontsize=18, rotation=0)
    plt.plot(X, y, "b.")    
    plt.plot(x1, y_pred, "r.-", linewidth=2, label="$\hat y$")

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# первый рисунок
plt.sca(axes[0])
plot_regression_predictions(tree_reg1, X, y)
plt.legend(loc='upper center', fontsize=18)
plt.title('max_depth=1', fontsize=14)
# второй рисунок
plt.sca(axes[1])
plot_regression_predictions(tree_reg2, X, y)
plt.title('max_depth=2', fontsize=14)
plt.ylabel('')

In [ ]:
tree_reg3 = DecisionTreeRegressor(max_depth=3)
tree_reg4 = DecisionTreeRegressor(max_depth=4)
tree_reg3.fit(X, y)
tree_reg4.fit(X, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# первый рисунок
plt.sca(axes[0])
plot_regression_predictions(tree_reg3, X, y)
plt.legend(loc='upper center', fontsize=18)
plt.title('max_depth=3', fontsize=14)
# второй рисунок
plt.sca(axes[1])
plot_regression_predictions(tree_reg4, X, y)
plt.title('max_depth=4', fontsize=14)
plt.ylabel('')

In [ ]:
tree_reg5 = DecisionTreeRegressor(max_depth=5)
tree_reg6 = DecisionTreeRegressor()
tree_reg5.fit(X, y)
tree_reg6.fit(X, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# первый рисунок
plt.sca(axes[0])
plot_regression_predictions(tree_reg5, X, y)
plt.legend(loc='upper center', fontsize=18)
plt.title('max_depth=5', fontsize=14)
# второй рисунок
plt.sca(axes[1])
plot_regression_predictions(tree_reg6, X, y)
plt.title('max_depth=15', fontsize=14)
plt.ylabel('')

In [ ]:
tree_reg6.get_depth()   # глубина

### Переобучение и регуляризация
Деревья решений почти не делают предположений о тренировочных данных (в отличие от, например, линейных моделей, которые предполагают, что данные являются линейными). Если не ставить ограничения, дерево подстраивается к данным как можно ближе, приводя к переобучению. Такие модели часто называются **непараметрическими моделями**, не от того, что в модели нет параметров (их может быть даже много), а от того, что число этих параметров заранее не определено, и структура модели может свободно прилегать к данным. В противоположность этому, **параметрические модели**, такие как линейные модели, имеют предопределенное число параметров, так что степень свободы ограничена, что снижает риск переобучения (хотя повышает риск недообучения).<br>


**Переобучение** (*overfitting*) - это проблема в машинном обучении, когда модель слишком точно подстраивается под тренировочные данные вместо того, чтобы находить общие закономерность, которая позволила бы прогнозировать результат на новых данных, которых она ранее не видела. При переобучении модель становится слишком сложной и начинает запоминать шум и случайности в тренировочных данных. Как результат, модель производит очень низкую ошибку на тренировочных данных, но плохо справляется с предсказанием на новых данных.

**Регуляризацией** называются методы, которые применяются для предотвращения переобучения, которые помогают ограничить сложность модели и улучшить ее обобщающую способность.

В случае с деревьями решений есть ряд регуляризирующих гиперпараметров, позволяющих избежать переобучение путем ограничения степеней свободы дерева решений. Регуляризующие гиперпараметры дерева решений зависят от алгоритма построения, но во всяком случае есть возможность ограничить глубину дерева гиперпараметром `max_depth` (по умолчанию стоит значение `None`, что означает неограниченность). <br>
Классы `DecisionTreeClassifier` и `DecisionTreeRegressor` имеет несколько других гиперпараметров, ограничивающих форму дерева:<br>
`min_samples_leaf` - минимальное число экземпляров в узле, необходимых для дальнейшего разбиения<br>
`min_weight_leaf` - минимальное число экземпляров, допустимых в узле<br>
`max_leaf_nodes` - максимальное число листов<br>
`max_features` - максимальное число признаков, оцениваемых при разделении узла<br>
Увеличение **min** гиперпараметров и уменьшение **max** гиперпараметров регуляризирует модель.

In [ ]:
tree_reg7 = DecisionTreeRegressor(max_depth=7)  
tree_reg7_reg1 = DecisionTreeRegressor(max_depth=7, min_samples_leaf=10)
tree_reg7.fit(X, y)
tree_reg7_reg1.fit(X, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# первый рисунок
plt.sca(axes[0])
plot_regression_predictions(tree_reg7, X, y)
plt.legend(loc='upper center', fontsize=18)
plt.title('No restriction', fontsize=14)
# второй рисунок
plt.sca(axes[1])
plot_regression_predictions(tree_reg7_reg1, X, y)
plt.title('min_samples_leaf=10', fontsize=14)
plt.ylabel('')

In [ ]:
tree_reg7_reg2 = DecisionTreeRegressor(max_depth=7, max_leaf_nodes=12)
tree_reg7_reg2.fit(X, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# первый рисунок
plt.sca(axes[0])
plot_regression_predictions(tree_reg7, X, y)
plt.legend(loc='upper center', fontsize=18)
plt.title('No restriction', fontsize=14)
# второй рисунок
plt.sca(axes[1])
plot_regression_predictions(tree_reg7_reg2, X, y)
plt.title('min_samples_leaf=10', fontsize=14)
plt.ylabel('')